# [Title]

## 1. Introduction

### 1.1 Literature review of the compound (<500 words)

### 1.2 Literature review of the cell factory (<500 words)

## 2. Problem definition (<300 words)

## 3. *If Project category I:* Reconstruction of a new GSM for your cell factory host of interest (<1500 words)

or

## 3. *If Project category II:* Selection and assessment of existing GSM (<500 words)

As previously mentioned, the model used for this assignment was provided by Imam et al. (2015) as an improvement of the previous model iRC1080. After completing the model, the authors actually conducted several experiments to contrast the accuracy of the new model's predictions.

A collection of 306 different genotype-phenotype conditions, obtained from 81 different mutants, was listed, and iCre1335 was used to predict the phenotypes of these mutants. The model showed a sensitivity of 83%, and a specificity of 92%, overall achieving an area under the Receiver Operating Curve (ROC) of 0.92. This means that only in 17% of the cases, the expression of a phenotype was predicted incorrectly; and in 8% of the cases, the lack of expression of a phenotype was predicted incorrectly.

Authors further tested the model by comparing the predicted growth rates under different metabolic conditions and C, N and P uptake rates with experimental cultures, obtaining a good agreement ($R^{2}$ = 0.82) between experimental and predicted values. Moreover, the model actually was able to predict growth halt and lipid accumulation due to nitrogen starvation, but unfortunately, it was unable to correctly predict the effect of light intensity changes in growth rate. In this case, the authors had to manually tune the photon uptake rate to match predictions with experimental results.

Overall, the authors were fairly confident of the predictive capabilities of the model. In order to independently verify that statement, we conducted our own validation, by uploading the file to memote, to analyse the consistency of the model. The report obtained, which has been added to this project's repository, shows that the model was quite satisfactory. Almost the totality of the reactions are both mass and charge balanced, there are almost no disconnected metabolite (meaning that they are not used in any reaction) and most of the reactions are not able to carry unlimited amounts of flux. The main problems are the lack of stoichiometric consistency, meaning that it is not possible to assign to each metabolite a positive molecular mass without breaking mass conservation at some point in the model (this is probably caused by the few reactions that are not mass balanced); and the fact that all metabolites, reactions and genes do not have any reference (annotation) linking them to one major database.

In [4]:
!memote report snapshot iCre1355_hetero_V2.xml

============================= test session starts ==============================
platform linux -- Python 3.6.12, pytest-6.1.2, py-1.9.0, pluggy-0.13.1
rootdir: /usr/local/lib/python3.6/dist-packages/memote/suite/tests
collected 164 items / 1 skipped / 163 selected                                 

../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_annotation.py F [  0%]
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF..         [ 39%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_basic.py . [ 40%]
.....F........F.....FF                                                   [ 53%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_biomass.py . [ 54%]
FFFF.FF.FFFFFFF...FFFFFFFFF                                              [ 70%]
../../../../usr/local/lib/python3.6/dist-packages/memote/suite/tests/test_consistency.py F [ 71%]
FFssssssssssssssss.FFFFFFFF.                                             [ 88%]

## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project; <500 words for Category I project)

# Model upload and general information

Firstly, the model given by the authors was loaded into the script, but it creates problems during the reading, due to formating of the data (conflicts with SBML definitions).

In [162]:
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

In [163]:
hetero = read_sbml_model('iCre1355_hetero.xml')
auto = read_sbml_model('iCre1355_auto.xml')
mixo = read_sbml_model('iCre1355_mixo.xml')

# Source: https://github.com/baliga-lab/Chlamy_model_iCre1355.git

cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre10.g451950.t1.2 OR Cre06.g284700.t1.2 )' for '<Reaction AAT at 0x7f63a99614e0>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre02.g143200.t1.1 OR ( Cre08.g368900.t1.2 OR Cre08.g368900.t2.1 ) )' for '<Reaction ALATLm at 0x7f63a99609e8>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre09.g390986.t1.1 OR Cre06.g279150.t1.2 )' for '<Reaction ASPTLm at 0x7f63a995ecf8>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre17.g747297.t1.1 OR Cre02.g076600.t1.2 )' for '<Reaction ALATAH at 0x7f63a995e940>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre17.g747297.t1.1 OR Cre02.g076600.t1.2 )' for '<Reaction ASNTAH at 0x7f63a995e0b8>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre17.g747297.t1.1 OR Cre02.g076600.t1.2 )' for '<Reaction ASPTAH at 0x7f63a995cf98>'
cobra/core/reacti

Therefore, in order to avoid constant warning messages that slow down computation, after the model is read for the first time, a new model is written with the correct format

In [164]:
write_sbml_model(hetero, 'iCre1355_hetero_V2.xml')
write_sbml_model(auto, 'iCre1355_auto_V2.xml')
write_sbml_model(mixo, 'iCre1355_mixo_V2.xml')

In [165]:
hetero = read_sbml_model('iCre1355_hetero_V2.xml')
auto = read_sbml_model('iCre1355_auto_V2.xml')
mixo = read_sbml_model('iCre1355_mixo_V2.xml')

In [166]:
hetero.name = 'iCre1355_Hetero'
auto.name = 'iCre1355_Auto'
mixo.name = 'iCre1355_Mixo'

models = [auto, mixo, hetero]
bio_growth = [auto.reactions.Biomass_Chlamy_auto, mixo.reactions.Biomass_Chlamy_mixo, hetero.reactions.Biomass_Chlamy_hetero]

Let´s now take a first look at the models. Let´s begin by checking the number of reactions, metabolites and genes:

In [167]:
for model in models:
    print('============================\n', model.name, '\n----------------------------')
    print('{0:22}  {1}'.format('Number of metabolites:', len(model.metabolites)))
    print('{0:22}  {1}'.format('Number of reactions:', len(model.reactions)))
    print('{0:22}  {1}'.format('Number of genes:', len(model.genes)))
    print('\n')

 iCre1355_Auto 
----------------------------
Number of metabolites:  1845
Number of reactions:    2394
Number of genes:        1963


 iCre1355_Mixo 
----------------------------
Number of metabolites:  1845
Number of reactions:    2394
Number of genes:        1963


 iCre1355_Hetero 
----------------------------
Number of metabolites:  1845
Number of reactions:    2394
Number of genes:        1963




Now, let's analyze other useful information, like the number of compartments, the media composition and the available exchange reactions

In [172]:
# Compartments

for model in models:
    print('============================\n', model.name, '\n----------------------------')
    for compound in model.medium:
        print('{0:15}  {1}'.format(compound, model.medium[compound]))
    print('\n')

 iCre1355_Auto 
----------------------------
EX_h_e           10.0
EX_h2o_e         10.0
EX_pi_e          10.0
EX_nh4_e         1.0
EX_so4_e         10.0
EX_fe2_e         10.0
EX_mg2_e         10.0
EX_na1_e         10.0
EX_photonVis_e   80.0
EX_o2_e          10.0
EX_co2_e         2.0


 iCre1355_Mixo 
----------------------------
EX_h_e           10.0
EX_h2o_e         10.0
EX_pi_e          10.0
EX_nh4_e         1.0
EX_so4_e         10.0
EX_fe2_e         10.0
EX_mg2_e         10.0
EX_na1_e         10.0
EX_photonVis_e   80.0
EX_o2_e          10.0
EX_co2_e         2.0
EX_ac_e          2.0


 iCre1355_Hetero 
----------------------------
EX_h_e           10.0
EX_h2o_e         10.0
EX_pi_e          10.0
EX_nh4_e         0.5
EX_so4_e         10.0
EX_fe2_e         10.0
EX_mg2_e         10.0
EX_na1_e         10.0
EX_o2_e          10.0
EX_ac_e          2.0




In [169]:
# Media

for model in models:
    print('============================\n', model.name, '\n----------------------------')
    for compartment in model.compartments:
        print('{0:2}  {1}'.format(compartment, model.compartments[compartment]))
    print('\n')

 iCre1355_Auto 
----------------------------
c   Cytosol
h   Chloroplast
m   Mitochondria
x   Glyoxysome
f   Flagellum
e   Extra-organism
n   Nucleus
g   Golgi Apparatus
s   Eyespot
u   Thylakoid Lumen
i   Inner Mitochondrial membrane space


 iCre1355_Mixo 
----------------------------
c   Cytosol
h   Chloroplast
m   Mitochondria
x   Glyoxysome
f   Flagellum
e   Extra-organism
n   Nucleus
g   Golgi Apparatus
s   Eyespot
u   Thylakoid Lumen
i   Inner Mitochondrial membrane space


 iCre1355_Hetero 
----------------------------
c   Cytosol
h   Chloroplast
m   Mitochondria
x   Glyoxysome
f   Flagellum
e   Extra-organism
n   Nucleus
g   Golgi Apparatus
s   Eyespot
u   Thylakoid Lumen
i   Inner Mitochondrial membrane space




In [170]:
# Available exchange reactions

for model in models:
    print('==========================================================================\n', model.name, '\n--------------------------------------------------------------------------')
    for exchange in model.exchanges:
        print('{0:38}  {1:15}  {2}'.format(exchange.name, exchange.id, exchange.reaction))
    print('\n')

 iCre1355_Auto 
--------------------------------------------------------------------------
H+ exchange                             EX_h_e           h_e <=> 
H2O exchange                            EX_h2o_e         h2o_e <=> 
Phosphate exchange                      EX_pi_e          pi_e <=> 
ammonia exchange                        EX_nh4_e         nh4_e <=> 
nitrate exchange                        EX_no3_e         no3_e --> 
sulfate exchange                        EX_so4_e         so4_e <=> 
Fe2+ exchange                           EX_fe2_e         fe2_e <=> 
Fe3+ exchange                           EX_fe3_e         fe3_e --> 
magnesium exchange                      EX_mg2_e         mg2_e <=> 
sodium exchange                         EX_na1_e         na1_e <=> 
photon emission                         EX_photonVis_e   photonVis_e <=> 
O2 exchange                             EX_o2_e          o2_e <=> 
CO2 exchange                            EX_co2_e         co2_e <=> 
HCO3 exchange          

Finally, let`s run the models and see the growth rate for each of them, as well as $H_{2}$ production rate when the model is set to maximize biomass growth and when the model is set to maximize $H_{2}$ productivity

In [171]:
for model, reaction in zip(models, bio_growth):
    model.objective = reaction
    with model:
        print('======================================================================\n', model.name, '\n----------------------------------------------------------------------')
        print(model.objective, '\n')
        print('{0:30}  {1}'.format('Biomass maximum growth rate:', model.optimize().objective_value))
        print('{0:30}  {1}'.format('Hydrogen productivity:', model.optimize().fluxes.EX_h2_e), '\n')        
        model.objective = model.reactions.EX_h2_e
        print('......................................................................')
        print(model.objective, '\n')
        print('{0:30}  {1}'.format('Biomass growth rate:', model.optimize().fluxes.get(reaction.id)))
        print('{0:30}  {1}'.format('Hydrogen maximum productivity:', model.optimize().objective_value), '\n\n\n\n')

 iCre1355_Auto 
----------------------------------------------------------------------
Maximize
1.0*Biomass_Chlamy_auto - 1.0*Biomass_Chlamy_auto_reverse_6a2df 

Biomass maximum growth rate:    0.0515751096837908
Hydrogen productivity:          -0.0 

......................................................................
Maximize
1.0*EX_h2_e - 1.0*EX_h2_e_reverse_f55e9 

Biomass growth rate:            9.300611804565978
Hydrogen maximum productivity:  0.0 




 iCre1355_Mixo 
----------------------------------------------------------------------
Maximize
1.0*Biomass_Chlamy_mixo - 1.0*Biomass_Chlamy_mixo_reverse_90004 

Biomass maximum growth rate:    0.15158877721813943
Hydrogen productivity:          0.0 

......................................................................
Maximize
1.0*EX_h2_e - 1.0*EX_h2_e_reverse_f55e9 

Biomass growth rate:            11.887320739055957
Hydrogen maximum productivity:  0.0 




 iCre1355_Hetero 
---------------------------------------------------

# Data visualization

In order to have a better general idea of how the model works, a package like escher can be used. Escher helps visualizing the methabolic pathways, but it only contains models for yeast, E. coli and humans, not for all microorganisms.

Nevertheless, a E. coli map can be loaded with Chlamydomomas reactions, so only the common pathways will be contain fluxes (reactions that are only present in Chlamydomomas will however not be shown in the model)

In [61]:
import escher
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [65]:
escher.Builder('iJO1366.Central metabolism',
               reaction_data=model.optimize().fluxes.to_dict()).display_in_notebook()

Unfortunately, this approach was less useful than expected, as most of the reactions present in the model (´iJO1366.Central metabolism´ is one of the most complete E. coli models) were silent, meaning that they are not present in the Chlamydomomas model.

# Genetic optimization

Prior to any modification to the model, let´s look for the reactions that produce the desired compound:

In [102]:
name = []
ID = []

for metabolite in model.metabolites.query('H2', 'name'):
    
    print(metabolite, '\t', metabolite.name)    
    if metabolite.name == 'H2':
        name.append(metabolite.name)
        ID.append(metabolite.id)
        
print(name, ID)

h2_c 	 H2
h2_e 	 H2
h2_h 	 H2
h2_m 	 H2
h2o_c 	 H2O
h2o_e 	 H2O
h2o_f 	 H2O
h2o_h 	 H2O
h2o_m 	 H2O
h2o_n 	 H2O
h2o_s 	 H2O
h2o_u 	 H2O
h2o_x 	 H2O
['H2', 'H2', 'H2', 'H2'] ['h2_c', 'h2_e', 'h2_h', 'h2_m']


In [109]:
reactions = []

for element in ID:
    compound = model.metabolites.get_by_id(element)
    for react in compound.reactions:
        reactions.append(react.id)

for element in reactions:
    print(model.reactions.get_by_id(element))

['HYDA', 'H2th', 'H2ti', 'H2tm', 'EX_h2_e', 'H2ti', 'HYDAh', 'H2th', 'HYDAm', 'H2tm']
HYDA: 2.0 fdxrd_c <=> 2.0 fdxox_c + h2_c + 2.0 h_c
H2th: h2_c <=> h2_h
H2ti: h2_c --> h2_e
H2tm: h2_c <=> h2_m
EX_h2_e: h2_e --> 
H2ti: h2_c --> h2_e
HYDAh: 2.0 fdxrd_h <=> 2.0 fdxox_h + h2_h + 2.0 h_h
H2th: h2_c <=> h2_h
HYDAm: 2.0 fdxrd_m <=> 2.0 fdxox_m + h2_m + 2.0 h_m
H2tm: h2_c <=> h2_m


We can see that the reactions that actually produce hydrogen are HYDA, HYDAh and HYDAm, the other ones just reflect transport of hydrogen between different compartments. Both HYDA, HYDAh and HYDAm are in fact the same reaction, the reason why it is splitted into three is because it can take place in three different compartments.

Now, let´s see what the maximum production of $H_{2}$ in the wild strain is, and which growth is achieved when hydrogen production is maximized: (ADD CODE FROM ANNETTE)

Let's try to enhance the production of hydrogen by OptKnock first:

In [ ]:
from cobra import Reaction, Metabolite
from cameo.strain_design.heuristic.evolutionary_based import OptGene
from cameo.strain_design.deterministic.linear_programming import OptKnock

with model:
    optknock = OptKnock(model, fraction_of_optimum=0.1)
    result = optknock.run(max_knockouts=1, target="ALATLm", biomass="Biomass_Chlamy_hetero")

In [128]:
with model:
    medium = model.medium
    medium['EX_lac_D_e'] = 100
    model.medium = medium
    model.objective = model.reactions.ALATLm
    optgene = OptGene(model)
    result = optgene.run(target=model.reactions.ALATLm, 
                         biomass=model.reactions.Biomass_Chlamy_hetero,
                         substrate=model.metabolites.lac_D_e,
                         max_evaluations=20, population_size=20, max_knockouts=5,
                         plot=False, growth_coupled=True)

Starting optimization at Sat, 14 Nov 2020 15:57:52


/home/jovyan/.local/lib/python3.6/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:286 RuntimeWarning: invalid value encountered in double_scalars
/home/jovyan/.local/lib/python3.6/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:286 RuntimeWarning: invalid value encountered in double_scalars
/home/jovyan/.local/lib/python3.6/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:286 RuntimeWarning: invalid value encountered in double_scalars
/home/jovyan/.local/lib/python3.6/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:286 RuntimeWarning: invalid value encountered in double_scalars
/home/jovyan/.local/lib/python3.6/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:286 RuntimeWarning: invalid value encountered in double_scalars
/home/jovyan/.local/lib/python3.6/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions

KeyboardInterrupt: 

## 5. Discussion (<500 words)

## 6. Conclusion (<200 words)

## References

What has to be done:
    
    Create exchange reactions for the lipids (otherwise they would accumulate, breaking the assumption of steady state)
    
    Reduce biomass growth in order to allow carbon to be diverted into lipid production pathways (otherwise it would not be driven towards this pathways, as it would be a lost of  C for growth) and to simulate N shortage (even though that would trigger a lot of genome expression / silention, as that is a stres condition in which cells are struggling to keep alive)
    
    Set the objecive function to the sum of the exchange reactions of lipids (optimize the total sum)
    
    Use escher with the map of E.coli, but with the reactions and fluxes of C.reinhardtii (some reactions wont be present in escher, others present in escher will have no flux at all)